In [3]:
import requests
import time
import pandas as pd
from datetime import date, timedelta
from tqdm.auto import tqdm

# ─── CONFIG ─────────────────────────────────────────────────────────────────────
API_KEY    = "2Xm1jWjTkGdQlDyTSGTsWFvNga_Tpaak"
BASE_URL   = "https://api.polygon.io/v2/reference/news"
TICKERS    = ["AAPL","GOOGL","TSLA"]
START_DATE = date(2023, 6, 1)
END_DATE   = date(2025, 6, 1)
OUTPUT_CSV = "../data/processed/news_data_polygon.csv"
# ────────────────────────────────────────────────────────────────────────────────

records = []
delta = timedelta(days=1)

for single_day in tqdm(pd.date_range(START_DATE, END_DATE), desc="Days"):
    day_str = single_day.date().isoformat()  # "YYYY-MM-DD"
    for ticker in TICKERS:
        params = {
            "ticker": ticker,
            "published_utc.gte": day_str,
            "published_utc.lte": day_str,
            "limit": 100,
            "apiKey": API_KEY
        }
        resp = requests.get(BASE_URL, params=params)
        resp.raise_for_status()
        for news in resp.json().get("results", []):
            records.append({
                "ds": pd.to_datetime(news["published_utc"]).normalize(),
                "Ticker": ticker,
                "headline": news.get("title", news.get("description",""))
            })
        time.sleep(12)   # ~5 calls per minute => 12 s pause

# Save to disk
news_df = pd.DataFrame(records)
news_df.to_csv(OUTPUT_CSV, index=False)
print(f"\nFetched {len(news_df)} headlines over {news_df['ds'].nunique()} unique days.")


Days:  30%|███       | 221/732 [2:18:33<5:16:18, 37.14s/it]

In [54]:
OUTPUT_CSV = "../data/processed/news_data_polygon.csv"
news_df.to_csv(OUTPUT_CSV, index=False)
print(f"\nFetched {len(news_df)} headlines over {news_df['ds'].nunique()} unique days.")


Fetched 19114 headlines over 729 unique days.
